In [1]:
import sys
sys.path.append('../')
import polars as pl
from sklearn.linear_model import LinearRegression
import numpy as np
from base.impactpy.model.subroutines import simple_regression

In [2]:
n_groups = 5
n_sample_groups = 3
n = 100
n_samples = n * n_sample_groups * n_groups

beta = 10
alpha = 3

x = np.random.normal(0, 1, n_samples)
eps = np.random.normal(0, 1, n_samples)
y = beta * x + alpha + eps

lf = pl.LazyFrame({'x': x, 'y': y})
df = (
	lf.with_row_index('idx')
	.with_columns(
		group=pl.col('idx').mod(n_groups) + 1,
		sample=pl.col('idx').mod(n_sample_groups) + 1
	)
).collect()

In [3]:
df

shape: (1_500, 5)
┌──────┬───────────┬───────────┬───────┬────────┐
│ idx  ┆ x         ┆ y         ┆ group ┆ sample │
│ ---  ┆ ---       ┆ ---       ┆ ---   ┆ ---    │
│ u32  ┆ f64       ┆ f64       ┆ u32   ┆ u32    │
╞══════╪═══════════╪═══════════╪═══════╪════════╡
│ 0    ┆ 2.447509  ┆ 27.052581 ┆ 1     ┆ 1      │
│ 1    ┆ -0.380678 ┆ 0.278339  ┆ 2     ┆ 2      │
│ 2    ┆ -1.291858 ┆ -9.776055 ┆ 3     ┆ 3      │
│ 3    ┆ 0.221278  ┆ 6.362562  ┆ 4     ┆ 1      │
│ 4    ┆ 0.301168  ┆ 6.293297  ┆ 5     ┆ 2      │
│ …    ┆ …         ┆ …         ┆ …     ┆ …      │
│ 1495 ┆ 0.363293  ┆ 5.981465  ┆ 1     ┆ 2      │
│ 1496 ┆ 1.746525  ┆ 18.416512 ┆ 2     ┆ 3      │
│ 1497 ┆ -0.419311 ┆ -0.473673 ┆ 3     ┆ 1      │
│ 1498 ┆ -0.503391 ┆ -0.97189  ┆ 4     ┆ 2      │
│ 1499 ┆ -0.772036 ┆ -4.45353  ┆ 5     ┆ 3      │
└──────┴───────────┴───────────┴───────┴────────┘

In [4]:
reg = LinearRegression()
reg.fit(x.reshape(-1, 1), y.reshape(-1, 1))

LinearRegression()

In [5]:
print(f'{reg.coef_=}')
print(f'{reg.intercept_=}')
print(f'{reg.score(x.reshape(-1, 1), y.reshape(-1, 1))=}')

reg.coef_=array([[10.00913289]])
reg.intercept_=array([3.00232995])
reg.score(x.reshape(-1, 1), y.reshape(-1, 1))=0.9901788582864927


In [6]:
res_df = simple_regression(df, 'x', 'y', 'group', 'sample').sort(['group', 'sample'])

In [7]:
import pandas as pd

groups = list(range(1, n_groups + 1))
samples = list(range(1, n_sample_groups + 1))

test_df = pd.DataFrame(
	index=pd.MultiIndex.from_product([groups, samples], names=['group', 'sample']),
	columns=['beta', 'alpha', 'rsq'],
)
reg = LinearRegression()
for group in groups:
	subdf = df.filter(pl.col('group').eq(group) & pl.col('sample').eq(samples[0]))
	reg.fit(subdf.select('x'), subdf.select('y'))
	for sample in samples:
		subdf = df.filter(pl.col('group').eq(group) & pl.col('sample').eq(sample))
		test_df.loc[group, sample] = [reg.coef_[0][0], reg.intercept_[0], reg.score(subdf.select('x'), subdf.select('y'))]
test_df = pl.DataFrame(test_df.reset_index())
res_df = res_df.select(['group', 'sample', 'beta', 'alpha', 'rsq'])

In [8]:
test_df

shape: (15, 5)
┌───────┬────────┬───────────┬──────────┬──────────┐
│ group ┆ sample ┆ beta      ┆ alpha    ┆ rsq      │
│ ---   ┆ ---    ┆ ---       ┆ ---      ┆ ---      │
│ i64   ┆ i64    ┆ f64       ┆ f64      ┆ f64      │
╞═══════╪════════╪═══════════╪══════════╪══════════╡
│ 1     ┆ 1      ┆ 10.022273 ┆ 3.238573 ┆ 0.991532 │
│ 1     ┆ 2      ┆ 10.022273 ┆ 3.238573 ┆ 0.983495 │
│ 1     ┆ 3      ┆ 10.022273 ┆ 3.238573 ┆ 0.993304 │
│ 2     ┆ 1      ┆ 10.06876  ┆ 2.863743 ┆ 0.991937 │
│ 2     ┆ 2      ┆ 10.06876  ┆ 2.863743 ┆ 0.988514 │
│ …     ┆ …      ┆ …         ┆ …        ┆ …        │
│ 4     ┆ 2      ┆ 10.05574  ┆ 3.023145 ┆ 0.989066 │
│ 4     ┆ 3      ┆ 10.05574  ┆ 3.023145 ┆ 0.982936 │
│ 5     ┆ 1      ┆ 9.972347  ┆ 3.058336 ┆ 0.993017 │
│ 5     ┆ 2      ┆ 9.972347  ┆ 3.058336 ┆ 0.990253 │
│ 5     ┆ 3      ┆ 9.972347  ┆ 3.058336 ┆ 0.986986 │
└───────┴────────┴───────────┴──────────┴──────────┘

In [9]:
res_df

shape: (15, 5)
┌───────┬────────┬───────────┬──────────┬──────────┐
│ group ┆ sample ┆ beta      ┆ alpha    ┆ rsq      │
│ ---   ┆ ---    ┆ ---       ┆ ---      ┆ ---      │
│ u32   ┆ u32    ┆ f64       ┆ f64      ┆ f64      │
╞═══════╪════════╪═══════════╪══════════╪══════════╡
│ 1     ┆ 1      ┆ 10.022273 ┆ 3.238573 ┆ 0.991532 │
│ 1     ┆ 2      ┆ 10.022273 ┆ 3.238573 ┆ 0.983495 │
│ 1     ┆ 3      ┆ 10.022273 ┆ 3.238573 ┆ 0.993304 │
│ 2     ┆ 1      ┆ 10.06876  ┆ 2.863743 ┆ 0.991937 │
│ 2     ┆ 2      ┆ 10.06876  ┆ 2.863743 ┆ 0.988514 │
│ …     ┆ …      ┆ …         ┆ …        ┆ …        │
│ 4     ┆ 2      ┆ 10.05574  ┆ 3.023145 ┆ 0.989066 │
│ 4     ┆ 3      ┆ 10.05574  ┆ 3.023145 ┆ 0.982936 │
│ 5     ┆ 1      ┆ 9.972347  ┆ 3.058336 ┆ 0.993017 │
│ 5     ┆ 2      ┆ 9.972347  ┆ 3.058336 ┆ 0.990253 │
│ 5     ┆ 3      ┆ 9.972347  ┆ 3.058336 ┆ 0.986986 │
└───────┴────────┴───────────┴──────────┴──────────┘

In [10]:
from polars.testing import assert_frame_equal

In [11]:
assert_frame_equal(test_df, res_df, check_dtype=False)